In [ ]:
#1スケール40変数完全モデル・完全観測でのETKF
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import random


# Simulation parameters
t0 = 0.0
dt = 1.e-2
assimilation_interval = 5#12  # 同化間隔
n_timestep = assimilation_interval * 4 * 365  # 合計タイムステップ
n_data = n_timestep // assimilation_interval  # データポイント数
km = 40#8  # slow variables
jm = 0
# kmjm = km * jm  # fast variables
F_ex = 20.0
m = 100 #n of ensemble member
#Assimilation parameters
rho=1.0

# Constants for the two-way coupled system(not used)
h = 0
b = 10.0
c = 0



# Load binary data for true values
with open("1scale_t_truedata_fort_50span.bin", "rb") as f_t:
    data_time = np.frombuffer(f_t.read(), dtype=np.float64)

print(data_time)


with open("1scale_xtrue_timeseries_km40_F8_50span.bin", "rb") as f_x1:
    true_data_x = np.frombuffer(f_x1.read(), dtype=np.float64)

true_data_x = true_data_x.reshape((n_data, km))



len_x = km
len_obs = km


In [ ]:
@njit(cache=True)
def model_forecast(t,X,km):
    model = np.empty(km)  # Initialize the model array
    for k in range(km):
        model[k] =  X[(k - 1) % km] * (X[(k + 1) % km] - X[(k - 2) % km]) - X[k] + F_ex
    return model


@njit(cache=True)
def model_nature(t,X,km,jm): #km:1scale目の変数の数 kmjm:2scale目の変数の数
    kmjm = km*jm
    len_X=len(X)
    X1 = X[:km]
    X2 = X[km:]
    model = np.zeros(len_X)  # Initialize the model array
    dX1 = np.zeros(km)
    dX2 = np.zeros(kmjm)

    for k in range(km):
        dX1[k] = X1[(k - 1) % km] * (X1[(k + 1) % km] - X1[(k - 2) % km]) - X1[k] + F_ex - h * c / b * np.sum(X2[jm * k : jm * (k + 1)])
    for j in range(kmjm):
        dX2[j] = -c * b * X2[(j+1) % kmjm] * (X2[(j + 2) % kmjm] - X2[(j - 1) % kmjm]) - c * X2[j] + h * c / b * X1[(j // jm)]
    model[:km] = dX1
    model[km:] = dX2
    return model


@njit(cache=True)
def RK4(t,model, y0, dt, steps, *model_args):
    """
    4次のルンゲクッタ法で微分方程式を数値積分する汎用関数。

    Parameters:
        model (function): 微分方程式モデル f(t, y, *args) を指定。
                          t (float): 現在の時間
                          y (array-like): 現在の状態ベクトル
                          *args: モデルに渡す追加の引数
        y0 (array-like): 初期状態ベクトル
        dt (float): 時間刻み幅
        steps (int): 積分するステップ数
        *model_args: モデルに渡す追加の引数（任意）

    Returns:
        t (numpy.ndarray): 時間の配列
        y (numpy.ndarray): 各時間ステップの状態ベクトル
    """
    # y0 = np.array(y0)
    # y = np.zeros((steps + 1, len(y0)))
    # t = np.zeros(steps + 1)
    # y[0] = y0
    # t[0] = t0

    y = y0
    t = t0

    for i in range(steps):
        k1 = dt * model(t, y, *model_args)
        k2 = dt * model(t + dt / 2, y + k1 / 2, *model_args)
        k3 = dt * model(t + dt / 2, y + k2 / 2, *model_args)
        k4 = dt * model(t + dt, y + k3, *model_args)


        y = y + (k1 + 2 * k2 + 2 * k3 + k4) / 6
        t = t + dt
    return  y



@njit(cache=True)
def compute_R_atr(true_data):
    n = true_data.shape[0]
    R_atr = 0.0
    for k in range(n):
        for l in range(k + 1, n):  # l_listに相当
            diff = true_data[k] - true_data[l]
            R_atr += np.sqrt(np.sum(diff ** 2))
    R_atr = R_atr * 2.0 / (n * (n - 1))
    return R_atr


In [ ]:
# Generate observation data by adding noise to true data
rng = np.random.default_rng(2357)
obs_data_x = true_data_x + rng.standard_normal(true_data_x.shape) ##ここだった!
# obs_data_x = true_data_x + rng.standard_normal(len_x) ##ここだった!



# Identity matrices for H and R
H = np.identity(len_x)
R = np.identity(len_x)
H_inv = np.linalg.inv(H)
R_inv = np.linalg.inv(R)



# Analysis and forecast data arrays
analyzed_data = np.zeros((n_data, len_x)) #(時系列数, 成分数)
forecast_data = np.zeros((n_data, len_x))

# Initial values for analysis and forecast
#アトラクタ上の値から解析値の初期値を出す
with open("1scale_x_initial_km40_F8.bin", "rb") as i_x1:
    init_data = np.frombuffer(i_x1.read(), dtype=np.float64)

n_init = 1000
init_data = init_data.reshape((n_init,km)) #true_data_x.reshape((n_data, km))

print('init_data', init_data.shape)

R_atr = compute_R_atr(init_data)

# R_atr = compute_R_atr(true_data_x)

print('R_atr=',R_atr)

In [ ]:
analyzed = np.zeros((len_x,m))
forecast = np.zeros((len_x,m))
d_forecast = np.zeros((len_x,m)) #(成分数,アンサンブル数)
d_obs = np.zeros((len_x,m))
init_idx = list(np.arange(0,n_init))

# while len(init_idx) < 2*m:
#     init_idx.add(int(rng.integers(0,n_init)))



# init_idx = list(init_idx)
random.seed(1234)
random.shuffle(init_idx)
print('initial indices for analyzed',init_idx[:m])
# print('initial indices for forecast',init_idx[m:2*m])

ens=0
for ens in range(m): #ensambleの作成
    random_idx_anlz = init_idx[ens]
    analyzed[:,ens] = init_data[random_idx_anlz,:] #use climatology as initial ensamble
    # analyzed[:,ens] = true_data[0,:] +rng.standard_normal(len_x) #use t0's data as inital ensamble

    # random_idx_fcst = init_idx[m+ens]
    # forecast[:,ens] =init_data[random_idx_fcst,:] #+rng.standard_normal(km) * 1.e-2 #+ pert[ens]
    # forecast[:,ens] =obs_data_x[0,:] +rng.standard_normal(km)*1.e-1 #not necessary
    forecast[:,ens] =analyzed[:,ens] +rng.standard_normal(len_x)  #not necessary




analyzed_data[0,:] = np.mean(analyzed,axis=1)
forecast_data[0, :] = np.mean(forecast,axis=1)
print('analyzed[0]', analyzed[0])
# Trace arrays
trace_P_a = np.zeros(n_data)
trace_P_f = np.zeros(n_data)
spread = np.zeros(n_data)

P_a = np.identity(len_x) * R_atr
trace_P_a[0] = np.trace(P_a)
spread[0] = R_atr

e=0
for e in range(m):
    d_forecast[:,e] = forecast[:,e] - forecast_data[0,:]

P_f = d_forecast @d_forecast.T/(m-1)
trace_P_f[0] = np.trace(P_f)

# Main loop for assimilation
# print(analyzed[:,0].shape)

assim_time = np.zeros(n_data)
assim_time[0] = data_time[0] #同化サイクルの時間発展

ddT = np.zeros((len_x,len_x)) #use in adaptive multiplicative inf.


# for i in range(1, 100):
# for i in range(1):
i=0
for i in range(1, n_data):
    # Forecast step using RK4
    e=0
    for e in range(m):
        forecast[:,e] = RK4(assim_time[i-1],model_forecast, analyzed[:,e], dt, assimilation_interval, km) #i-1回目の解析値からi回目の予報値を計算

    forecast_data[i] = np.mean(forecast,axis=1)#.reshape((km,1))

#compute X_b & Y_b
    e=0
    for e in range(m):
        d_forecast[:,e] = (forecast[:,e] - forecast_data[i,:])#.reshape((km,1))
        d_obs[:,e] = H @ d_forecast[:,e] #(観測数,アンサンブル数) #linear observation
        # d_obs[:,e] = H @ d_forecast[:,e].T #(観測数,アンサンブル数)

        # d_obs[:,e] = H @ forecast[:,e] - H @ forecast_data[i] #(観測数,アンサンブル数)


    # print('d_obs',d_obs, d_obs.shape)



    # #Adaptive multiplicative inflation
    # increment= np.empty((len_obs,m))
    e=0
    # for e in range(m):
    #     increment[:,e] = (obs_data_x[i] - H @ forecast[:,e]) #H:(l_x,l_x)

    # ddT_past = ddT.copy()
    # ddT = np.zeros((len_x,len_x))
    e=0
    # for e in range(m):
    #     incre = increment[:,e].reshape((len_obs,1))
    #     ddT += incre @ incre.T

    # ddT = ddT/(m)

    # if i > 2:
    #     # rho =   np.trace(H_inv @ (ddT - R) @ H_inv.T) /np.trace(d_forecast @d_forecast.T/(len_x-1)) #P_f = d_forecast @d_forecast.T/(len_x-1)
    #     rho = (np.trace((ddT * R_inv)) - R.size) / np.trace(H @ d_forecast @d_forecast.T/(len_x-1) @ H.T * R_inv)
    # print('rho',rho)


    #localization

    C = d_obs.T @ np.linalg.inv(R) #(観測数,アンサンブル数).T@(観測数,観測数)→(アンサンブル数,観測数)


    l,U = np.linalg.eig((((m-1) /rho) * np.identity(m)) + C @ d_obs)
    D_1 = np.diag(l**-1.)
    # P_a = U @ D_1 @ U.T  #P~a = UD^(-1)U.T

    # D_2 = np.diag(np.sqrt(1./l))#ｌ^-0.5を対角成分にもつ行列D^-0.5の生成
    D_2 = np.diag(l**-0.5)#ｌ^-0.5を対角成分にもつ行列D^-0.5の生成

    dWa =  U @ D_2 @U.T

    dWa = dWa * (m-1)**0.5   #いわゆるT(変換行列)
    Pa_tild = dWa @ dWa /(m-1)  #P_a_tilder

    P_f = d_forecast @d_forecast.T/(m-1)
    P_a = d_forecast@Pa_tild@d_forecast.T


    # # RTPP
    # alpha = 0.96
    # dWa = dWa * alpha + np.identity(m) * (1.-alpha)

    # # RTPS
    # alpha = 0.97
    # sigma_fcst = (np.trace(P_f)/(m-1))**0.5
    # sigma_anlz = (np.trace(P_a)/(m-1))**0.5
    # # print('tr(P_a)',np.trace(P_a))
    # # print('sigma_anlz',sigma_anlz)
    # dWa *=   ((1.- alpha) * sigma_fcst +  alpha * sigma_anlz)/sigma_anlz

    # P_a = dWa @ dWa /(m-1) #インフレーションを適用して再計算

    ##not use eigenvalue decomp.
    # P_a = np.linalg.inv((m-1)/rho * np.identity(m) + C @ d_obs)
    # dWa = scipy.linalg.sqrtm((m-1)*P_a)
    # print('dWa-dWa.T',(dWa-dWa.T).max())



    wbar_a = Pa_tild @ C @(obs_data_x[i] - H @ forecast_data[i]) #(m,m)@(m,km)@(km,1) = (m,1)

    W = dWa
    e=0
    for e in range(m):
        W[:,e] += wbar_a
        analyzed[:,e] = forecast_data[i] + d_forecast @ W[:,e]


    analyzed_data[i] = np.mean(analyzed, axis=1)


#RTPS
    d_analyzed =(analyzed.T - analyzed_data[i]).T
    alpha =0.
    alpha_param=1.0
    for n in range(km):
        d_analyzed[n,:] *= 1.- alpha + alpha * np.linalg.norm(d_forecast[n,:])/np.linalg.norm(d_analyzed[n,:])
    for n in range(km, len_x):
        d_analyzed[n,:] *= 1. - alpha_param + alpha_param * np.linalg.norm(d_forecast[n,:])/np.linalg.norm(d_analyzed[n,:])
    analyzed = (analyzed_data[i] + d_analyzed.T).T

    P_a = d_analyzed@d_analyzed.T/(m-1)


    if i <=1:
        print('analyzed', analyzed.shape)
        print('forecast_data['+str(i)+']', forecast_data[i].shape)
        print('analyzed_data['+str(i)+']', analyzed_data[i].shape)
        print('forecast[0,0]',forecast[0,0])


    # analyzed_data[i,:] = forecast_data[i,:] + d_forecast@P_a@d_obs.T@np.linalg.inv(R)@(obs_data_x[i]-H@forecast_data[i,:])

    trace_P_f[i] = np.trace(P_f)
    trace_P_a[i] = np.trace(P_a) #Pa = Xb @Pa_tild @ Xb.T
    spread[i] = np.mean([np.linalg.norm(analyzed[:,_]-analyzed.mean(axis=1) )for _ in range(m)])/len_x**0.5
    assim_time[i] = assim_time[i-1] + dt * assimilation_interval


In [ ]:
np.abs(np.sqrt(P_f[n,n]/P_a[n,n]))

In [ ]:
np.abs((P_f[n,n]/P_a[n,n]))**0.5

In [ ]:
P_f[n,n]

In [ ]:
P_a[n,n]

In [ ]:
# Compute RMSE
RMSE_obs_x1 = np.linalg.norm(obs_data_x[:,:km] - true_data_x[:,:km], axis=1) / km ** 0.5
RMSE_forecast_x1 = np.linalg.norm(forecast_data[:,:km] - true_data_x[:,:km], axis=1) / km ** 0.5
RMSE_analysis_x1 = np.linalg.norm(analyzed_data[:,:km] - true_data_x[:,:km], axis=1) / km ** 0.5


In [ ]:
# Plot results
fig1=plt.figure(figsize=(8,4))
ax1 = fig1.add_subplot(111)
# spread=(trace_P_a/(m-1))**0.5
ax1.plot((data_time-data_time[0])/(dt*assimilation_interval), RMSE_obs_x1, label="Observation RMSE")
ax1.plot((assim_time-assim_time[0])/(dt*assimilation_interval), RMSE_forecast_x1, label="Forecast RMSE: "+str(np.mean(RMSE_forecast_x1)))
ax1.plot((assim_time-assim_time[0])/(dt*assimilation_interval), RMSE_analysis_x1, label="Analysis RMSE: "+str(np.mean(RMSE_analysis_x1)), alpha=1.)
ax1.plot((assim_time-assim_time[0])/(dt*assimilation_interval), spread, label='Spread: '+str(np.mean(spread)))
ax1.axhline(0.2, color='k', linestyle='--', label="0.2")

ax1.set_xlabel("Assimilation step")
# ax1.set_ylim((-0.5,1.5))
ax1.legend()
ax1.set_title('RMSEs (ETKF)')

plt.show()
print(np.mean(RMSE_analysis_x1[1000:]))

In [ ]:

# Plot results
fig2 = plt.figure()
ax2 = fig2.add_subplot(312)
ax2.plot((assim_time-assim_time[0])/(dt*assimilation_interval), trace_P_a, label='tr(P$_a$)')

# ax2.axhline(0.2, color='red', linestyle='--', label="0.2")

ax2.set_xlabel("Timestep")
ax2.set_ylim((0,10))
ax2.legend()

# Plot results
ax3 = fig2.add_subplot(313)
ax3.plot((assim_time-assim_time[0])/(dt*assimilation_interval), trace_P_f, label='tr(P$_f$)')

# ax2.axhline(0.2, color='red', linestyle='--', label="0.2")

ax3.set_xlabel("Timestep")
ax3.set_ylim((-0,10))
ax3.legend()

plt.show()


In [ ]:
Tend =200
# Plot results
fig2 = plt.figure(figsize=(14,4))
ax2 = fig2.add_subplot(121)
ax2.plot((assim_time[:Tend]-assim_time[0])/(dt*assimilation_interval), forecast_data[:Tend,0], label='Forecast', color='r', linestyle='--')

ax2.plot((assim_time[:Tend]-assim_time[0])/(dt*assimilation_interval), true_data_x[:Tend,0], label='True', color='lime')

ax2.scatter((assim_time[:Tend]-assim_time[0])/(dt*assimilation_interval), obs_data_x[:Tend,0], label='Observation', color='b', s=5)
ax2.set_ylabel("x$_{1}$")
ax2.set_xlabel("Assimilation Timestep")

ax2.legend()

# Plot results
ax3 = fig2.add_subplot(122)
# Plot results
ax3.plot((data_time[:Tend]-data_time[0])/(dt*assimilation_interval), RMSE_obs_x1[:Tend], label="Observation", color='b')
ax3.plot((assim_time[:Tend]-assim_time[0])/(dt*assimilation_interval), RMSE_forecast_x1[:Tend], label="Forecast", color='r')

# ax2.axhline(0.2, color='red', linestyle='--', label="0.2")

ax3.set_xlabel("Assimilation Timestep")
ax3.set_ylabel("RMSE")


ax3.set_ylim((-0,5))
ax3.legend()
fig2.suptitle('Data assimilation with ETKF')
plt.show()


In [ ]:
# # Parameter survey
# rho_values = np.arange(1.0, 1.51, 0.01)  # rhoを1.0から1.5まで0.01刻み
# n_rho = len(rho_values)  # rhoの値の個数

# # ヒートマップ用のRMSE配列を初期化
# RMSE_heatmap = np.zeros((n_rho, n_data))

# for idx, rho in enumerate(rho_values):
#     # rhoを変更してシミュレーションを実行
#     analyzed_data = np.zeros((n_data, len_x))
#     forecast_data = np.zeros((n_data, len_x))

#     # 初期値設定（コード内から流用）
#     analyzed = np.zeros((len_x, m))
#     forecast = np.zeros((len_x, m))
#     for ens in range(m):
#         analyzed[:km, ens] += 0 + rng.standard_normal(km)
#         forecast[:, ens] = obs_data_x[0, :]

#     analyzed_data[0, :] = np.mean(analyzed, axis=1)
#     forecast_data[0, :] = np.mean(forecast, axis=1)

#     for i in range(1, n_data):
#         for e in range(m):
#             forecast[:, e] = RK4(assim_time[i-1], model_forecast, analyzed[:, e], dt, assimilation_interval, km)

#         forecast_data[i] = np.mean(forecast, axis=1)

#         for e in range(m):
#             d_forecast[:, e] = forecast[:, e] - forecast_data[i]
#             d_obs[:, e] = H @ forecast[:, e] - H @ forecast_data[i]

#         C = d_obs.T @ np.linalg.inv(R)
#         l, U = np.linalg.eig((m-1)/rho * np.identity(m) + C @ d_obs)
#         D_1 = np.diag(1./l)
#         P_a = U @ D_1 @ U.T
#         D_2 = np.diag(np.sqrt(1./l))
#         W_a = U @ D_2 @ U.T * np.sqrt(m-1)
#         wbar_a = P_a @ C @ (obs_data_x[i] - H @ forecast_data[i])

#         for e in range(m):
#             W_a[e] += wbar_a
#             analyzed[:, e] = d_forecast @ W_a[e, :] + forecast_data[i]

#         analyzed_data[i] = np.mean(analyzed, axis=1)

#     RMSE_forecast_x1 = np.linalg.norm(forecast_data[:, :km] - true_data_x[:, :km], axis=1) / km ** 0.5
#     RMSE_heatmap[idx, :] = RMSE_forecast_x1

# # ヒートマップの描画
# plt.figure(figsize=(12, 6))
# plt.imshow(RMSE_heatmap, aspect='auto', extent=[0, n_data, 1.0, 1.5], origin='lower', cmap='viridis')
# plt.colorbar(label="RMSE_forecast_x1")
# plt.xlabel("Time step")
# plt.ylabel("rho")
# plt.title("Heatmap of RMSE_forecast_x1")
# plt.show()


In [ ]:
# np.save('./1scaleL96_RMSE_forecast_heatmap.npy',RMSE_heatmap)

In [ ]:
Tend =200
# Plot results
fig2 = plt.figure()
ax2 = fig2.add_subplot(111, projection='3d')

ax2.plot(true_data_x[Tend:,0],true_data_x[Tend:,14],true_data_x[Tend:,24])


ax2.set_ylabel("x$_{1}$")
ax2.set_ylabel("x$_{15}$")
ax2.set_ylabel("x$_{25}$")


In [ ]:
# # RMSE_forecast_x1 の最小値を持つ rho を計算
# min_rmse_idx = np.unravel_index(np.argmin(RMSE_heatmap), RMSE_heatmap.shape)
# rho_min_rmse = rho_values[min_rmse_idx[0]]

# # RMSE_forecast_x1[1:] の平均を最小化する rho を計算
# mean_rmse_per_rho = np.mean(RMSE_heatmap[:, :-1000], axis=1)  # 時系列ラスト100個の平均
# rho_min_mean_rmse = rho_values[np.argmin(mean_rmse_per_rho)]

# # ヒートマップの描画
# plt.figure(figsize=(16, 9))
# plt.imshow(RMSE_heatmap, aspect='auto', extent=[0, n_data, 1.0, 1.5], origin='lower', cmap='viridis')
# plt.colorbar(label="RMSE_forecast_x1")
# plt.xlabel("Time step")
# plt.ylabel("rho")
# plt.title("Heatmap of RMSE_forecast_x1")

# # 最小 RMSE の位置に hline を追加
# plt.axhline(y=rho_min_rmse, color='red', linestyle='--', label=f"Min RMSE rho = {rho_min_rmse:.2f}")

# # 平均 RMSE を最小化する rho の位置に hline を追加
# plt.axhline(y=rho_min_mean_rmse, color='blue', linestyle='--', label=f"Min Avg RMSE rho = {rho_min_mean_rmse:.2f}")

# # 凡例を追加
# plt.legend(loc="upper right")

# plt.show()


In [ ]:
# def ETKF(i, analyzed_pre, model,y0, dt, steps, *model_args):
#     # for i in range(1, n_data):
#     # # for i in range(1):
#     #     # Forecast step using RK4
#     #     for e in range(m):
#     #         forecast[:,e] = RK4(assim_time[i-1],model_forecast, analyzed[:,e], dt, assimilation_interval, km) #i-1回目の解析値からi回目の予報値を計算

#     #     forecast_data[i] = np.mean(forecast,axis=1)#.reshape((km,1))


#     #     for e in range(m):
#     #         d_forecast[:,e] = (forecast[:,e] - forecast_data[i])#.reshape((km,1))
#     #         # d_obs[:,e] = H @ forecast[:,e] - H @ forecast_data[i] #(観測数,アンサンブル数)
#     #         d_obs[:,e] = H @ forecast[:,e] - H @ forecast_data[i] #(観測数,アンサンブル数)


#     #     # print('d_obs',d_obs, d_obs.shape)
#     #     # print('d_obs.T @ np.linalg.inv(R) @ d_obs', (d_obs.T @ np.linalg.inv(R) @ d_obs).shape)


#     #     # #Adaptive multiplicative inflation
#     #     # increment= np.empty((len_obs,m))
#     #     # for e in range(m):
#     #     #     increment[:,e] = (obs_data_x[i] - H @ forecast[:,e]) #H:(l_x,l_x)

#     #     # ddT_past = ddT.copy()
#     #     # ddT = np.zeros((len_x,len_x))

#     #     # for e in range(m):
#     #     #     incre = increment[:,e].reshape((len_obs,1))
#     #     #     ddT += incre @ incre.T

#     #     # ddT = ddT/(m)

#     #     # if i > 2:
#     #     #     # rho =   np.trace(H_inv @ (ddT - R) @ H_inv.T) /np.trace(d_forecast @d_forecast.T/(len_x-1)) #P_f = d_forecast @d_forecast.T/(len_x-1)
#     #     #     rho = (np.trace((ddT * R_inv)) - R.size) / np.trace(H @ d_forecast @d_forecast.T/(len_x-1) @ H.T * R_inv)
#     #     # print('rho',rho)


#     #     #localization



#     #     C = d_obs.T @ np.linalg.inv(R)
#     #     l,U = np.linalg.eig((m-1)/rho * np.identity(m) + C @ d_obs)

#     #     D_1 = np.diag(1./l)
#     #     P_a = U @ D_1 @ U.T  #P~a = UD^(-1)U.T
#     #     P_f = d_forecast @d_forecast.T/(len_x-1)

#     #     # D_2 = np.diag(1./np.sqrt(l))#ｌ^-0.5を対角成分にもつ行列D^-0.5の生成
#     #     D_2 = np.diag(np.sqrt(1./l))#ｌ^-0.5を対角成分にもつ行列D^-0.5の生成

#     #     W_a =  U @ D_2 @U.T * np.sqrt(m-1)

#     #     wbar_a = P_a @ C @(obs_data_x[i] - H @ forecast_data[i])

#     #     for e in range(m):
#     #         # W_a[:,e] += wbar_a  #(m,m)行列の各列に(m)ベクトルを足す
#     #         W_a[e] += wbar_a  #(m,m)行列の各行に(m)ベクトルを足す　たぶんこっちが正しい

#     #         analyzed[:,e] = d_forecast @ W_a[e,:] + forecast_data[i]
#     #         # analyzed[:,e] = (d_forecast @ W_a[e,:]).T  + forecast_data[i] #転置とってみた



#     #     # l,U = np.linalg.eig((m-1)/rho * np.identity(m) + d_obs.T @ np.linalg.inv(R) @ d_obs)
#     #     # print('l[0]:',l[0])
#     #     # l[0] = np.mean(l[1:])
#     #     # print('l[0]:',l[0])
#     #     # D_2 = np.diag(np.sqrt(1./l))#ｌ^-0.5を対角成分にもつ行列D^-0.5の生成
#     #     # D_2 = np.diag(1./np.sqrt(l))#ｌ^-0.5を対角成分にもつ行列D^-0.5の生成

#     #     # analyzed =  d_forecast @ (P_a @ d_obs.T @ np.linalg.inv(R) @ (obs_data_x[i] - np.mean(H @ forecast, axis=1)) + np.sqrt(m-1)*U @ D_2 @ U.T) #(成分数, アンサンブル数)


#     #     # analyzed += forecast_data[i,:].reshape((72,1))

#     #     analyzed_data[i] = np.mean(analyzed, axis=1)
